In [280]:
%cd /home/rana/Projects/zindi
import yaml
with open('common/config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
checkpoint = config.get('checkpoint')
image_name = config.get('image_name')
file_path = config.get('file_path')
ct_model_path=config.get('ct_model_path')+checkpoint

with open(file_path, 'r') as file:
    yaml_content = yaml.safe_load(file)
    yaml_content['services']['translation_inference_util']['image'] = image_name

# Save the updated YAML content back to the file
    with open(file_path, 'w') as file:
        yaml.dump(yaml_content, file, default_flow_style=False)

/home/rana/Projects/zindi


/home/rana/.cache/pypoetry/virtualenvs/zindi-z3yfXQo9-py3.9/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [281]:
ct_model_path

'models/nllb/checkpoint-618'

In [282]:
!rm -rf deployment/saved_model/
!mkdir deployment/saved_model/
!cp -r {ct_model_path} deployment/saved_model/{checkpoint}/

In [283]:
%cd deployment

/home/rana/Projects/zindi/deployment


In [284]:
%%writefile requirements.txt
# transformers
# sacremoses>=0.1.1
pyyaml>=6.0.2
kserve>=0.13.1
ctranslate2==4.3.1
# sentencepiece==0.1.99
# kserve==0.11.2
# torch>=2.4.0
# accelerate = "^0.33.0"
sentencepiece>=0.2.0

Overwriting requirements.txt


In [285]:
# filename = "main.py"
# with open(filename, "w") as file:
#     file.write(f"""
# \"\"\"
# KServe inference script for NLLB-200 translation model.
# \"\"\"

# import argparse
# import os
# from typing import List
# from kserve import (InferOutput, InferRequest, InferResponse, Model, ModelServer, model_server)
# from kserve.utils.utils import generate_uuid
# #from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import ctranslate2
# import sentencepiece as spm
# import re
# import sys
# import unicodedata
# from sacremoses import MosesPunctNormalizer

# # Constants
# MODEL_DIR = "./saved_model/{checkpoint}"

# def get_non_printing_char_replacer(replace_by: str = " "):
#     non_printable_map = {{
#         ord(c): replace_by
#         for c in (chr(i) for i in range(sys.maxunicode + 1))
#         # same as \p{{C}} in perl
#         # see https://www.unicode.org/reports/tr44/#General_Category_Values
#         if unicodedata.category(c) in {{"C", "Cc", "Cf", "Cs", "Co", "Cn"}}
#     }}

#     def replace_non_printing_char(line) -> str:
#         return line.translate(non_printable_map)

#     return replace_non_printing_char

# replace_nonprint = get_non_printing_char_replacer(" ")

# class TranslationModel(Model):
#     \"\"\"
#     KServe inference implementation of NLLB-200 translation model.
#     \"\"\"

#     def __init__(self, name: str):
#         \"\"\"
#         Initialize the translation model.
#         Args:
#             name (str): Name of the model.
#         \"\"\"        
#         super().__init__(name)
#         self.name = name
#         self.ready = False
#         self.model = None
#         self.tokenizer = None
#         # self.sp_source_model = None
#         # self.sp_target_model = None
#         self.mpn=None
#         self.load()

#     def load(self) -> None:
#         \"\"\"
#         Load model and tokenizer from disk.
#         \"\"\"
#         try:
#             self.tokenizer = spm.SentencePieceProcessor(model_file=MODEL_DIR+'/sentencepiece.bpe.model')
#             self.model = ctranslate2.Translator(MODEL_DIR)
#             self.mpn = MosesPunctNormalizer(lang="fr")
#             self.mpn.substitutions = [(re.compile(r), sub) for r, sub in self.mpn.substitutions]
#             print('Model and tokenizer loaded')
#             self.ready = True
#         except Exception as e:
#             print('Error loading model: ', e)
#             self.ready = False

#     def preprocess(self, payload: InferRequest, *args, **kwargs) -> str:
#         \"\"\"
#         Preprocess inference request.

#         Args:
#             payload (InferRequest): The input payload containing the text to translate.

#         Returns:
#             str: Preprocessed text ready for translation.
#         \"\"\"

#         text = payload.inputs[0].data[0]
#         clean = self.mpn.normalize(text)
#         clean = replace_nonprint(clean)
#         clean = unicodedata.normalize("NFKC", clean)
#         return clean

#     def predict(self, data: str, *args, **kwargs) -> InferResponse:
#         \"\"\"
#         Make prediction using the model.
#         Args:
#             data (str): Preprocessed input text.

#         Returns:
#             InferResponse: KServe inference response containing the translated text.
#         \"\"\"
#         source_sentences = data.strip()
#         source_sentences = self.mpn.normalize(source_sentences)
#         source_sentences = replace_nonprint(source_sentences)
#         source_sentences = unicodedata.normalize("NFKC", source_sentences)
#         translation = self._translate(self.model, source_sentences)

#         return self._create_response(translation)
    

#     # Ctranslate2 translation
#     def _translate(self, model, text):
#         target_prefix = [['fra_Latn']] * len(text)
#         source_sents_subworded = [["dyu_Latn"] + self.tokenizer.encode_as_pieces(sent) + ["</s>"] for sent in [text]]
#         try:
#             translations= self.model.translate_batch(
#                     source_sents_subworded,
#                     batch_type="tokens",
#                     max_batch_size=256,
#                     beam_size=1,
#                     target_prefix=[["fra_Latn"]],
#                     return_scores=False,
#                     return_attention=False,
#                     return_alternatives=False,
#                 )
#             translation = translations[0].hypotheses[0]
#             if "fra_Latn" in translation:
#                 translation.remove("fra_Latn")
#             trans = self.tokenizer.decode(translation)
#         except Exception as e:
#             trans = ["Error: "+str(e)]  # Return empty string if translation fails
#         return trans

#     def _create_response(self, translation: str) -> InferResponse:
#         \"\"\"
#         Create InferResponse object.

#         Args:
#             translation (str): Translated text.

#         Returns:
#             InferResponse: KServe inference response object.
#         \"\"\"
#         return InferResponse(
#             model_name=self.name,
#             infer_outputs=[InferOutput(name=\"output-0\", shape=[1], datatype=\"STR\", data=[translation])],
#             response_id=generate_uuid()
#         )

# def parse_arguments() -> argparse.Namespace:
#     \"\"\"
#     Parse command-line arguments.

#     Returns:
#         argparse.Namespace: Parsed command-line arguments.
#     \"\"\"
#     parser = argparse.ArgumentParser(parents=[model_server.parser])
#     # Check if '--model_name' is already defined
#     model_name_defined = any('--model_name' in action.option_strings for action in model_server.parser._actions)

#     if not model_name_defined:
#         model_server.parser.add_argument(
#             '--model_name', 
#             default='model', 
#             help='The name that the model is served under.'
#         )
#     return parser.parse_args()

# def main():
#     \"\"\"
#     Main function to start the model server.
#     \"\"\"
#     args = parse_arguments()
#     model = TranslationModel(args.model_name)
#     ModelServer().start([model])

# if __name__ == "__main__":
#     main()

# """)

In [286]:
filename = "main.py"
with open(filename, "w") as file:
    file.write(f"""
\"\"\"
KServe inference script for NLLB-200 translation model.
\"\"\"

import argparse
import os
from typing import List
from kserve import (InferOutput, InferRequest, InferResponse, Model, ModelServer, model_server)
from kserve.utils.utils import generate_uuid
import ctranslate2
import sentencepiece as spm

# Constants
MODEL_DIR = "./saved_model/{checkpoint}"

class TranslationModel(Model):
    \"\"\"
    KServe inference implementation of NLLB-200 translation model.
    \"\"\"

    def __init__(self, name: str):
        \"\"\"
        Initialize the translation model.

        Args:
            name (str): Name of the model.
        \"\"\"
        super().__init__(name)
        self.name = name
        self.ready = False
        self.model = None
        self.tokenizer = None
        self.mpn = None
        self.load()

    def load(self) -> None:
        \"\"\"
        Load model and tokenizer from disk.
        \"\"\"
        try:
            self.tokenizer = spm.SentencePieceProcessor(model_file=os.path.join(MODEL_DIR, 'sentencepiece.bpe.model'))
            self.model = ctranslate2.Translator(
                MODEL_DIR,
                device="cpu",
                device_index=0,
                compute_type="int8",
                intra_threads=os.cpu_count(),
                inter_threads=1,
            )
            print('Model and tokenizer loaded')
            self.ready = True
        except Exception as e:
            print('Error loading model:', e)
            self.ready = False

    def preprocess(self, payload: InferRequest, *args, **kwargs) -> str:
        \"\"\"
        Preprocess inference request.

        Args:
            payload (InferRequest): The input payload containing the text to translate.

        Returns:
            str: Preprocessed text ready for translation.
        \"\"\"
        text = payload.inputs[0].data[0]
        return text.strip()

    def predict(self, data: str, *args, **kwargs) -> InferResponse:
        \"\"\"
        Make prediction using the model.

        Args:
            data (str): Preprocessed input text.

        Returns:
            InferResponse: KServe inference response containing the translated text.
        \"\"\"
        translation = self._translate(self.model, data)
        return self._create_response(translation)

    def _translate(self, model, text):
        \"\"\"
        Translate the input text using the ctranslate2 library.

        Args:
            model (ctranslate2.Translator): The translation model.
            text (str): The input text to be translated.

        Returns:
            str: The translated text.
        \"\"\"
        target_prefix = [['fra_Latn']] * len([text])
        source_sents_subworded = [["dyu_Latn"] + self.tokenizer.encode_as_pieces(sent) + ["</s>"] for sent in [text]]
        try:
            translations = self.model.translate_batch(
                source_sents_subworded,
                batch_type="tokens",
                max_batch_size=256,
                beam_size=1,
                target_prefix=target_prefix,
                return_scores=False,
                return_attention=False,
                return_alternatives=False,
            )
            translation = translations[0].hypotheses[0]
            if "fra_Latn" in translation:
                translation.remove("fra_Latn")
            trans = self.tokenizer.decode(translation)
        except Exception as e:
            trans = ["Error: " + str(e)]  # Return error message if translation fails
        return trans

    def _create_response(self, translation: str) -> InferResponse:
        \"\"\"
        Create InferResponse object.

        Args:
            translation (str): Translated text.

        Returns:
            InferResponse: KServe inference response object.
        \"\"\"
        return InferResponse(
            model_name=self.name,
            infer_outputs=[InferOutput(name="output-0", shape=[1], datatype="STR", data=[translation])],
            response_id=generate_uuid()
        )

def parse_arguments() -> argparse.Namespace:
    \"\"\"
    Parse command-line arguments.

    Returns:
        argparse.Namespace: Parsed command-line arguments.
    \"\"\"
    parser = argparse.ArgumentParser(parents=[model_server.parser])
    # Check if '--model_name' is already defined
    model_name_defined = any('--model_name' in action.option_strings for action in model_server.parser._actions)

    if not model_name_defined:
        model_server.parser.add_argument(
            '--model_name',
            default='model',
            help='The name that the model is served under.'
        )
    return parser.parse_args()

def main():
    \"\"\"
    Main function to start the model server.
    \"\"\"
    args = parse_arguments()
    model = TranslationModel(args.model_name)
    ModelServer().start([model])

if __name__ == "__main__":
    main()
""")

In [287]:
import getpass
import subprocess

def run_command(command_str):
    # Prompt for the sudo password
    sudo_password = getpass.getpass("Enter your sudo password: ")
    # Run the command with sudo, passing the password
    result = subprocess.run(f"echo {sudo_password} | sudo -S {command_str}", shell=True, check=True)
    # Check the result
    print(result)

In [288]:
run_command("docker compose down")

[sudo] password for rana:  Container translation_inference_ctutil  Stopping
 Container translation_inference_ctutil  Stopped
 Container translation_inference_ctutil  Removing
 Container translation_inference_ctutil  Removed
 Network deployment_default  Removing


CompletedProcess(args='echo 1234 | sudo -S docker compose down', returncode=0)


 Network deployment_default  Removed


In [289]:
run_command(f"docker build -t {image_name} .")

[sudo] password for rana: #0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 360B done
#1 DONE 0.2s

#2 [internal] load metadata for docker.io/library/python:3.10.14-slim
#2 DONE 2.8s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.2s

#4 [1/6] FROM docker.io/library/python:3.10.14-slim@sha256:8666a639a54acc810408e505e2c6b46b50834385701675ee177f578b3d2fdef9
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 466.17MB 4.8s
#5 transferring context: 633.53MB 6.8s done
#5 DONE 7.0s

#6 [2/6] WORKDIR /app
#6 CACHED

#7 [3/6] COPY ./requirements.txt .
#7 CACHED

#8 [4/6] RUN pip install --no-cache-dir -r requirements.txt
#8 CACHED

#9 [5/6] COPY ./saved_model /app/saved_model
#9 DONE 16.1s

#10 [6/6] COPY ./main.py /app/main.py
#10 DONE 2.1s

#11 exporting to image
#11 exporting layers
#11 exporting layers 4.3s done
#11 writing image sha256:6ad0e9834267443283

CompletedProcess(args='echo 1234 | sudo -S docker build -t zindi-image:0.3.1 .', returncode=0)


In [290]:
run_command(f"docker compose up -d")

[sudo] password for rana:  Network deployment_default  Creating
 Network deployment_default  Created
 Container translation_inference_ctutil  Creating
 Container translation_inference_ctutil  Created
 Container translation_inference_ctutil  Starting


CompletedProcess(args='echo 1234 | sudo -S docker compose up -d', returncode=0)


 Container translation_inference_ctutil  Started


In [291]:
!curl -X POST http://localhost:8080/v2/models/model/infer -H 'Content-Type: application/json' -d @/home/rana/Projects/zindi/common/input.json

{"model_name":"model","model_version":null,"id":"c60e699e-80fe-43e0-a90e-2e4b527611a8","parameters":null,"outputs":[{"name":"output-0","shape":[1],"datatype":"STR","parameters":null,"data":["Jean et son père connus."]}]}

In [270]:
run_command(f"sudo docker compose logs")

[sudo] password for rana: 

translation_inference_ctutil  | 2024-08-21 12:04:17.471 1 kserve INFO [model_server.py:register_model():384] Registering model: model
translation_inference_ctutil  | 2024-08-21 12:04:17.473 1 kserve INFO [model_server.py:start():254] Setting max asyncio worker threads as 5
translation_inference_ctutil  | 2024-08-21 12:04:17.476 1 kserve INFO [model_server.py:serve():260] Starting uvicorn with 1 workers
translation_inference_ctutil  | 2024-08-21 12:04:17.640 uvicorn.error INFO:     Started server process [1]
translation_inference_ctutil  | 2024-08-21 12:04:17.640 uvicorn.error INFO:     Waiting for application startup.
translation_inference_ctutil  | 2024-08-21 12:04:17.645 1 kserve INFO [server.py:start():63] Starting gRPC server on [::]:8081
translation_inference_ctutil  | 2024-08-21 12:04:17.646 uvicorn.error INFO:     Application startup complete.
translation_inference_ctutil  | 2024-08-21 12:04:17.647 uvicorn.error INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to q

In [292]:
!rm -rf ../submission
!mkdir ../submission
!mkdir ../submission/deployment
!cp Dockerfile ../submission/deployment/
!cp main.py ../submission/deployment/main.py
!cp requirements.txt ../submission/deployment/requirements.txt
# !cp image_name.txt ../submission/image_name.txt
!cp README.md ../submission/README.md

In [293]:
image_uri = "058264459447.dkr.ecr.eu-west-1.amazonaws.com/highwind/f463c7a9-f305-472f-8e49-5f602a7d7882/875c2cfb-bc5b-4234-8826-3326639bea62:latest"

In [294]:
%cd ../submission/
!rm -rf image_name.txt
with open('image_name.txt', 'w') as f:
    f.write(image_uri)
!zip -r submission.zip .

/home/rana/Projects/zindi/submission
  adding: image_name.txt (deflated 16%)
  adding: deployment/ (stored 0%)
  adding: deployment/requirements.txt (deflated 29%)
  adding: deployment/Dockerfile (deflated 29%)
  adding: deployment/main.py (deflated 67%)
  adding: README.md (stored 0%)
